In [1]:
import imblearn
import pandas as pd
import numpy as np
from sklearn import svm
import os

In [2]:
import gc
import math
from sklearn.utils import shuffle

In [3]:
os.environ["_CCTRACE_"]="ON";
import cloudComPy as cc;
cc.initCC();

QSocketNotifier: Can only be used with threads started with QThread
/home/paul/projets/CloudComPy/CloudComPy/CloudCompare/libs/qCC_db/src/ccLog.cpp [48] : trace ON
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [966] : initializeConverters
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [152] : register QString_from_python_str
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [237] : register Vector2Tpl_from_python_tuple
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [237] : register Vector2Tpl_from_python_tuple
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [291] : register Vector2Tpl_from_python_list
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [291] : register Vector2Tpl_from_python_list
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [345] : register Vector3Tpl_from_python_tuple
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converter

In [4]:
import sys
import time
import matplotlib.pyplot as plt
import joblib

In [5]:
from imblearn.under_sampling import RandomUnderSampler
from cloudComPy import CloudSamplingTools

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import plot_confusion_matrix

In [7]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from statistics import mean, stdev
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [8]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [9]:
# Data Preparation and cleaning Steps
# Combine data file with label and clean
# Calculate the Geometric Features for the data set

In [10]:
# Combining the labels and datasets
def combine_data(file):
    # Load data
    df = pd.read_csv('../Data/Third/'+file+'.txt', sep = ' ')
    label = pd.read_csv('../Data/Third/'+file+'.labels')
    # Drop the color columns
    df.columns = range(df.shape[1])
    df.drop(df.columns[[3, 4, 5, 6]], axis = 1, inplace = True)
    # Combine labels and datasets
    df[3] = label
    # Drop the rows with the Label 0
    #df.drop(df[4], axis=0, inplace =True)
    df.drop(df[(df[3] == 0)].index, inplace=True)
    #df.to_csv('../Data/Test/'+file+'_combined.txt', index=False)

    return df

In [11]:
# Splitting the datasets to allow balancing using the classes in the target field
def split_data(data):
    target = np.array(data[3])
    features = data.drop(data.columns[[3]], axis = 1)
    features = np.array(features)
    return target, features

In [12]:
def split_dataframe(data):
    target = np.array(data['Label'])
    features = data.drop(['Label'], axis = 1)
    features = np.array(features)
    return target, features

In [13]:
# Balancing the dataset using the target field
def bal_classes(feat, target,sel):
    if sel == 1:
        sm = SMOTE(random_state=42)
        X_r , y_r = sm.fit_resample(feat,target)
    else :
        us = RandomUnderSampler(sampling_strategy='not minority')
        X_r, y_r = us.fit_resample(feat, target)
    X_r = shuffle(X_r, random_state=42)
    y_r = shuffle(y_r, random_state=42)
    return X_r, y_r

In [14]:
# Recombining the target and features as a balanced dataset
def create_dataframe(features, target, name):
    df_temp = pd.DataFrame(data=features)
    df_temp[3] = target
    df_temp.to_csv('../Data/Test/'+name+'_combined.txt', index=False)

    return df_temp

In [15]:
# Data processing and cleaning

In [16]:
# Combine files with labels and clean
df = combine_data('bildstein_station3_xyz_intensity_rgb')

In [17]:
# Balance the Class labels
target, features  = split_data(df)
X_r, y_r = bal_classes(features, target,0)
print(target.shape)
print(features.shape)

(7890622,)
(7890622, 3)


In [18]:
# Convert to Dataframe and save for usage, ensure the dataset and the label 
# file are in the same folder. It saves the name and appends '_combined.txt' at the end.
df_temp = create_dataframe(X_r,y_r,'bildstein_station3_xyz_intensity_rgb')

In [19]:
df_temp.shape

(56320, 4)

In [ ]:
cloud = compute_features('bildstein_station5_xyz_intensity_rgb_combined')

In [20]:
# Calculating the Geometric Features for a given cloud

In [21]:
# Computing geometric features
def compute_features(data):
    cloud = cc.loadPointCloud('../Data/Test/'+data+'_combined.txt');
    # cord = cloud.toNpArrayCopy();
    clouds = [cloud]

    rad = cc.GetPointCloudRadius(clouds)
    size = cloud.size()  
    #scale_half = round(size/2)
    # scale_quater = round(size/4)
    # scale_tenth = round(size/10)
    #sub_sample_half = CloudSamplingTools.resampleCloudWithOctree(cloud,scale_half,cc.RESAMPLING_CELL_METHOD.CELL_CENTER)
    # sub_sample_quarter = CloudSamplingTools.resampleCloudWithOctree(cloud,scale_quater,cc.RESAMPLING_CELL_METHOD.CELL_CENTER )
    # sub_sample_tenth = CloudSamplingTools.resampleCloudWithOctree(cloud,scale_tenth,cc.RESAMPLING_CELL_METHOD.CELL_CENTER )

    print("Computing ")
    cc.computeFeature(cc.GeomFeature.Anisotropy,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenEntropy,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenValue1,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenValue2,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenValue3,rad,clouds)
    cc.computeFeature(cc.GeomFeature.EigenValuesSum,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Linearity ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Omnivariance ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.PCA1 ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.PCA2 ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Planarity ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Sphericity ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.SurfaceVariation ,rad,clouds)
    cc.computeFeature(cc.GeomFeature.Verticality ,rad,clouds)
    #cc.computeFeature(cc.GeomFeature.Verticality ,cc.GetPointCloudRadius(clouds),clouds)
    return cloud 

In [1]:
# Create a dataframe from a cloud input and return it
def cloud_to_dataframe(cloud_name):
    cloud = cc.loadPointCloud(cloud_name);
    cord = cloud.toNpArrayCopy();
    
    df = pd.DataFrame(cord, columns = ['x','y','z'])
    
    #fields_lists = ['Intensity','Label','Eigenvalues_sum','Omnivariance','Eigenentropy','Anisotropy','Planarity','Linearity','PCA1','PCA2','Surface_variation','Sphericity','Verticality','1st_eigenvalue','2nd_eigenvalue','3rd_eigenvalue']
    fields_lists = ['Label','Eigenvalues_sum','Omnivariance','Eigenentropy','Anisotropy','Planarity','Linearity','PCA1','PCA2','Surface_variation','Sphericity','Verticality','1st_eigenvalue','2nd_eigenvalue','3rd_eigenvalue']
    nsf = cloud.getNumberOfScalarFields()
    # Asign Columns names and values
    for i in range(nsf):
        df[fields_lists[i]] = cloud.getScalarField(i).toNpArrayCopy().tolist()
    # Drop rows with null values
    df.dropna(axis = 0,inplace = True)
    print(df['Label'].value_counts())
    return df

In [23]:
# Compute the geometric features of the file created previously, and use the cloud variable
cloud = compute_features('bildstein_station3_xyz_intensity_rgb')

# Returns a dataframe from a cloud.
temp_df = cloud_to_dataframe(cloud)

/home/paul/projets/CloudComPy/CloudComPy/pyAPI/pyCC.cpp [432] : Opening file: ../Data/Test/bildstein_station3_xyz_intensity_rgb_combined.txt mode: 0 x: 0 y: 0 z: 0 extraData: 
[CloudCompare] [ccColorScalesManager] Found 0 custom scale(s) in persistent settings
[CloudCompare] [I/O] File '../Data/Test/bildstein_station3_xyz_intensity_rgb_combined.txt' loaded successfully
/home/paul/projets/CloudComPy/CloudComPy/pyAPI/pyCC.cpp [596] : Found one cloud with 56321 points
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [

Computing 


756] : convertible to std::vector<ccHObject*>?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [759] : list ok
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [763] : nbobj 1
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [770] :   OK ccPointCloud0
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [794] : construct
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [810] :   Extract OK ccPointCloud0
/home/paul/projets/CloudComPy/CloudComPy/pyAPI/pyCC.cpp [804] : GetDefaultCloudKernelSize - NbPoints: 12 nbClouds: 1
/home/paul/projets/CloudComPy/CloudComPy/pyAPI/pyCC.cpp [808] : cloud name: bildstein_station3_xyz_intensity_rgb_combined - Cloud
/home/paul/projets/CloudComPy/CloudComPy/pyAPI/pyCC.cpp [818] : radius: 1.45905
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [756] : convertible to std::vector<ccHObject*>?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/conve

/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [159] : convertible to QString?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/ScalarFieldPy.cpp [42] : ScalarField ToNpArray with copy, ownership transfered to Python
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [159] : convertible to QString?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/ScalarFieldPy.cpp [42] : ScalarField ToNpArray with copy, ownership transfered to Python
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [159] : convertible to QString?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/ScalarFieldPy.cpp [42] : ScalarField ToNpArray with copy, ownership transfered to Python
/home/paul/projets/CloudComPy/CloudComPy/boostPython/converters.hpp [159] : convertible to QString?
/home/paul/projets/CloudComPy/CloudComPy/boostPython/ScalarFieldPy.cpp [42] : ScalarField ToNpArray with copy, ownership transfered to Python
/home/paul/projets/CloudComPy/Cl

8.0    7040
7.0    7040
4.0    7035
6.0    7005
2.0    6980
1.0    6904
5.0    6879
3.0    6654
Name: Label, dtype: int64


In [65]:
# Returns a dataframe from a cloud.
temp_df.to_csv('bildstein_station3_df')

In [67]:
# Combining the balanced dataframes to form one dataset
df1 = pd.read_csv('domfountain_station3_df')
df2 = pd.read_csv('bildstein_station1_df')
df3 = pd.read_csv('bildstein_station3_df')
df4 = pd.read_csv('untermaederbrunnen_station1_df')
frames = [df1, df2, df3, df4]
result = pd.concat(frames)


In [28]:
result.to_csv('composite_df', index = False)

[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=240; total time=   0.6s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   7.7s
[CV] END bootstrap=True, max_depth=70, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=240; total time=   0.8s
[CV] END bootstrap=True, max_depth=70, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=240; total time=   0.6s
[CV] END bootstrap=True, max_depth=70, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=240; total time=   0.7s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=680; total time=   1.7s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1560; total time=   4.3s
[CV] END bootstrap=False, max_depth=50, 

[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=240; total time=   0.6s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   7.4s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1780; total time=   9.0s
[CV] END bootstrap=False, max_depth=70, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=460; total time=   3.1s
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1780; total time=  11.4s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1780; total time=   9.6s
[CV] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=2000; total time=  12.4s
[CV] END bootstrap=True, max_dep

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time=   3.7s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1560; total time=   2.6s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=460; total time=   0.8s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1780; total time=   8.1s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=680; total time=   2.3s
[CV] END bootstrap=False, max_depth=70, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=460; total time=   3.3s
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1780; total time=  10.9s
[CV] END bootstrap=False, max_depth=

[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1120; total time=   2.0s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1560; total time=   2.7s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=680; total time=   1.2s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=460; total time=   0.8s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=900; total time=   3.7s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=   4.8s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=680; total time=   1.9s
[CV] END bootstrap=False, max_depth=7

In [16]:
result = pd.read_csv('composite_df')

In [26]:
result = result.drop(result.columns[0], axis = 1)

In [27]:
result.head()

,x,y,z,Label,Eigenvalues_sum,Omnivariance,Eigenentropy,Anisotropy,Planarity,Linearity,PCA1,PCA2,Surface_variation,Sphericity,Verticality,1st_eigenvalue,2nd_eigenvalue,3rd_eigenvalue
0,0.000,1.000,2.000,3.0,0.998217,0.166386,0.055349,0.000733,0.000099,0.056181,0.986758,0.001588,0.985197,0.013046,0.011459,0.001783,0.001757,0.250664
1,-1.668,10.590,1.119,7.0,0.993345,0.511124,0.227878,0.057523,0.001517,0.286918,0.747572,0.027090,0.794229,0.200485,0.245772,0.006655,0.005286,0.911689
2,-1.590,-1.266,-1.695,1.0,0.934994,0.795989,0.407738,0.221688,0.026505,0.655932,0.456297,0.133809,0.621616,0.337975,0.478697,0.065006,0.040409,0.049688
3,16.856,-21.368,-0.869,8.0,0.849596,0.709202,0.216074,0.131509,0.032498,0.380081,0.391369,0.097381,0.568494,0.346003,0.458227,0.150404,0.085504,0.495987
4,8.770,-4.213,-1.595,8.0,0.820106,0.665640,0.254039,0.064341,0.045700,0.364080,0.746726,0.090734,0.697755,0.176723,0.073380,0.179894,0.125522,0.853893


In [ ]:
# Data Modeling and Normalisation
# Initial Training 
# Hyperparameter search and Tunning 

In [29]:
# Split the dataframe 
target,features = split_dataframe(result)
# Apply Scaler Normalisation
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [ ]:
#  Initial Training 
# model = rfr()
# model.fit(features, target)
# # display the relative importance of each attribute
# print(model.feature_importances_)
#importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [18]:
# Spliting data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, target, test_size = 0.25, random_state = 42)

In [74]:
# Random Forest Classifier
rf = rfr(n_estimators = 1, random_state = 42, n_jobs = 4, warm_start = True,)
rf.fit(train_features, train_labels)
yhat_rf = rf.predict(test_features)
print(metrics.accuracy_score(test_labels, yhat_rf))

0.9805364546044855


In [ ]:
joblib.dump(rf, '../Project/rf_model.joblib')


In [ ]:
# SVM 
svc = svm.SVC(kernel = 'linear')
svc.fit(train_features, train_labels)
yhat_svm = svc.predict(test_features)
print(metrics.accuracy_score(test_labels, yhat_svm))

In [ ]:
joblib.dump(svc, '../Project/svc.joblib')


In [ ]:
# KNN Clustering
k= 3
KNN_model= KNeighborsClassifier(n_neighbors=k).fit(train_features, train_labels)
yhat_knn=KNN_model.predict(test_features)
#print(rf.score(yhat_rf, test_labels))
print(metrics.accuracy_score(test_labels, yhat_knn))

In [ ]:
joblib.dump(KNN_model, '../Project/knn.joblib')


In [ ]:
# Hyperparameter search and Tunning 

In [30]:
# Random Forest Tunning
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = rfr()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(train_features[0:400], train_labels[0:400])

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feat

/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feat

/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feat

/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feat

/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feat

/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feat

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [20, 240, 460, 680, 900,
                                                         1120, 1340, 1560, 1780,
                                                         2000]},
                   random_state=42, verbose=2)

In [31]:
# The best parameters for this type of data is 
rf_random.best_params_

{'n_estimators': 2000,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': True}

In [35]:
# Crossvalidation
# Random Forest
rf = rfr(n_estimators = 80, random_state = 42, n_jobs = 4, min_samples_split =2, min_samples_leaf=2, max_features = 'auto'
         , max_depth = 90, bootstrap = True, warm_start = True,)
rf_accu_stratified = []
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
X, y = features, target
skf.get_n_splits(X,y)
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #print("TRAIN:", train_index, "TEST:", test_index)
    rf.fit(X_train, y_train)
    rf_accu_stratified.append(rf.score(X_test, y_test))
    #print(rf.score(X_test, y_test))
    
    yhat_rf = rf.predict(X_test)
    #print(metrics.accuracy_score(y_test, yhat_rf))

print('\nMaximum Accuracy That can be obtained from this model is:',
      max(rf_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(rf_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(rf_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(rf_accu_stratified))

/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


KeyboardInterrupt: 

In [112]:
rf_best = rf
joblib.dump(rf_best, '../Project/rf_best.joblib')


['../Project/rf_best.joblib']

In [81]:
# Gridsearch Tunning SVM
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear','poly','sig']} 
  
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(train_features[0:4000], train_labels[0:4000])

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.667 total time=   0.7s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.679 total time=   0.7s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.700 total time=   0.7s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.679 total time=   0.7s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.710 total time=   0.7s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.884 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.897 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.897 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.891 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.905 total time=   0.1s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.938 total time=   0.3s
[CV 2/5] END .......C=0.1, gamma=1, kernel=pol

[CV 2/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.929 total time=   0.6s
[CV 3/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.922 total time=   0.7s
[CV 4/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.906 total time=   0.6s
[CV 5/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.911 total time=   0.6s
[CV 1/5] END .......C=1, gamma=1, kernel=linear;, score=0.909 total time=   0.1s
[CV 2/5] END .......C=1, gamma=1, kernel=linear;, score=0.917 total time=   0.1s
[CV 3/5] END .......C=1, gamma=1, kernel=linear;, score=0.912 total time=   0.1s
[CV 4/5] END .......C=1, gamma=1, kernel=linear;, score=0.900 total time=   0.1s
[CV 5/5] END .......C=1, gamma=1, kernel=linear;, score=0.917 total time=   0.1s
[CV 1/5] END .........C=1, gamma=1, kernel=poly;, score=0.940 total time=   0.8s
[CV 2/5] END .........C=1, gamma=1, kernel=poly;, score=0.936 total time=   0.8s
[CV 3/5] END .........C=1, gamma=1, kernel=poly;, score=0.950 total time=   0.9s
[CV 4/5] END .........C=1, g

[CV 4/5] END .........C=10, gamma=1, kernel=rbf;, score=0.909 total time=   0.6s
[CV 5/5] END .........C=10, gamma=1, kernel=rbf;, score=0.917 total time=   0.6s
[CV 1/5] END ......C=10, gamma=1, kernel=linear;, score=0.912 total time=   0.3s
[CV 2/5] END ......C=10, gamma=1, kernel=linear;, score=0.919 total time=   0.3s
[CV 3/5] END ......C=10, gamma=1, kernel=linear;, score=0.917 total time=   0.3s
[CV 4/5] END ......C=10, gamma=1, kernel=linear;, score=0.907 total time=   0.3s
[CV 5/5] END ......C=10, gamma=1, kernel=linear;, score=0.927 total time=   0.3s
[CV 1/5] END ........C=10, gamma=1, kernel=poly;, score=0.948 total time=   0.8s
[CV 2/5] END ........C=10, gamma=1, kernel=poly;, score=0.930 total time=   0.8s
[CV 3/5] END ........C=10, gamma=1, kernel=poly;, score=0.943 total time=   0.8s
[CV 4/5] END ........C=10, gamma=1, kernel=poly;, score=0.927 total time=   0.7s
[CV 5/5] END ........C=10, gamma=1, kernel=poly;, score=0.935 total time=   0.6s
[CV 1/5] END ...........C=10

[CV 1/5] END .....C=100, gamma=1, kernel=linear;, score=0.915 total time=   1.0s
[CV 2/5] END .....C=100, gamma=1, kernel=linear;, score=0.916 total time=   1.1s
[CV 3/5] END .....C=100, gamma=1, kernel=linear;, score=0.917 total time=   1.1s
[CV 4/5] END .....C=100, gamma=1, kernel=linear;, score=0.919 total time=   1.3s
[CV 5/5] END .....C=100, gamma=1, kernel=linear;, score=0.921 total time=   1.1s
[CV 1/5] END .......C=100, gamma=1, kernel=poly;, score=0.946 total time=   0.7s
[CV 2/5] END .......C=100, gamma=1, kernel=poly;, score=0.929 total time=   0.5s
[CV 3/5] END .......C=100, gamma=1, kernel=poly;, score=0.944 total time=   1.0s
[CV 4/5] END .......C=100, gamma=1, kernel=poly;, score=0.927 total time=   0.6s
[CV 5/5] END .......C=100, gamma=1, kernel=poly;, score=0.934 total time=   0.5s
[CV 1/5] END ..........C=100, gamma=1, kernel=sig;, score=nan total time=   0.0s
[CV 2/5] END ..........C=100, gamma=1, kernel=sig;, score=nan total time=   0.0s
[CV 3/5] END ..........C=100

[CV 3/5] END ....C=1000, gamma=1, kernel=linear;, score=0.919 total time=   8.0s
[CV 4/5] END ....C=1000, gamma=1, kernel=linear;, score=0.917 total time=   9.6s
[CV 5/5] END ....C=1000, gamma=1, kernel=linear;, score=0.924 total time=   8.2s
[CV 1/5] END ......C=1000, gamma=1, kernel=poly;, score=0.946 total time=   0.7s
[CV 2/5] END ......C=1000, gamma=1, kernel=poly;, score=0.929 total time=   0.5s
[CV 3/5] END ......C=1000, gamma=1, kernel=poly;, score=0.944 total time=   1.0s
[CV 4/5] END ......C=1000, gamma=1, kernel=poly;, score=0.927 total time=   0.6s
[CV 5/5] END ......C=1000, gamma=1, kernel=poly;, score=0.934 total time=   0.5s
[CV 1/5] END .........C=1000, gamma=1, kernel=sig;, score=nan total time=   0.0s
[CV 2/5] END .........C=1000, gamma=1, kernel=sig;, score=nan total time=   0.0s
[CV 3/5] END .........C=1000, gamma=1, kernel=sig;, score=nan total time=   0.0s
[CV 4/5] END .........C=1000, gamma=1, kernel=sig;, score=nan total time=   0.0s
[CV 5/5] END .........C=1000

/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
125 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
125 fits failed with the following error:
Traceback (most recent call last):
  File "/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9/site-packages/sklearn/svm/_base.py", line 251, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/home/fatsani/miniconda3/envs/CloudComPy39/lib/python3.9

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'poly', 'sig']},
             verbose=3)

In [82]:
print(grid.best_params_)


{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
SVC(C=100, gamma=0.1)


In [90]:
# Crossvalidation
# SVM using best parameters on a sample data
svc = svm.SVC(kernel = 'rbf', C = 1000, gamma = 0.01)
svc_accu_stratified = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X, y = features, target
skf.get_n_splits(X,y)
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #print("TRAIN:", train_index, "TEST:", test_index)
    svc.fit(X_train, y_train)
    svc_accu_stratified.append(rf.score(X_test, y_test))
    #print(rf.score(X_test, y_test))
    
    yhat_svc = svc.predict(X_test)
    print(metrics.accuracy_score(y_test, yhat_svc))

print('\nMaximum Accuracy That can be obtained from this model is:',
      max(svc_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(svc_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(svc_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(svc_accu_stratified))

0.9750943396226415
0.9720728370601001
0.9732993678648929
0.9744315501462402
0.9748089442400226

Maximum Accuracy That can be obtained from this model is: 100.0 %

Minimum Accuracy: 98.42173839199555 %

Overall Accuracy: 99.6843476783991 %

Standard Deviation is: 0.007058200483552152


In [111]:
svc_best = svc
joblib.dump(svc_best, '../Project/svc_best.joblib')


['../Project/svc_best.joblib']

In [91]:
# Hyperparameter search and Tuning KNN

In [104]:
estimator_KNN = KNeighborsClassifier(algorithm='auto')
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
weights = ['uniform', 'distance']
metric = ['minkowski', 'chebyshev']

hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p, weights = weights, metric= metric)
knn_ = KNeighborsClassifier()

# with GridSearch
grid = GridSearchCV(knn_, hyperparameters, cv=10)
grid.fit(train_features[0:400], train_labels[0:400])

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                       23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'metric': ['minkowski', 'chebyshev'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29],
                         'p': [1, 2], 'weights': ['uniform', 'distance']})

In [106]:

# examine the best model
print(grid.best_score_)
print(grid.best_params_)

0.8799999999999999
{'leaf_size': 1, 'metric': 'chebyshev', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}


In [107]:
#leaf_size': 1, 'metric': 'chebyshev', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform
# Crossvalidation
# KNN
knn= KNeighborsClassifier(n_neighbors=1, metric = 'chebyshev', p = 1, weights = 'uniform', leaf_size=1).fit(train_features, train_labels)
knn_accu_stratified = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X, y = features, target
skf.get_n_splits(X,y)
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #print("TRAIN:", train_index, "TEST:", test_index)
    knn.fit(X_train, y_train)
    knn_accu_stratified.append(knn.score(X_test, y_test))
    #print(rf.score(X_test, y_test))
    
    yhat_knn = knn.predict(X_test)
    print(metrics.accuracy_score(y_test, yhat_knn))

print('\nMaximum Accuracy That can be obtained from this model is:',
      max(knn_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(knn_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(knn_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(knn_accu_stratified))

0.975
0.9765072176620436
0.9775450514199453
0.975469383904142
0.976884611755826

Maximum Accuracy That can be obtained from this model is: 97.75450514199453 %

Minimum Accuracy: 97.5 %

Overall Accuracy: 97.62812529483914 %

Standard Deviation is: 0.0010383916580220343


In [109]:
knn_best = knn

In [110]:
joblib.dump(knn_best, '../Project/knn_best.joblib')


['../Project/knn_best.joblib']